In [ ]:
// Event hub stuff
var eventHubsNamespace = "evhns-timeseries-timeseries-u-001";
var eventHubsInstance = "received-timeseries";
var eventHubConnectionStringWithSharedAccessKeyAndEntityPath = "[INSERT EVENT HUB CONNECTION STRING]";

// Storage stuff
var storageAccountKey = "[INSERT STORAGE ACCOUNT KEY]";
var storageAccountName = "stdatalakesharedresu001";
var delta_lake_container_name = "timeseries-data";
var blobName = "timeseries-unprocessed";

In [ ]:
#r "nuget:Microsoft.Spark"
#r "nuget: Microsoft.Spark.Extensions.Delta, 2.1.0"
using System;
using System.Collections.Generic;
using Microsoft.Spark.Sql;
using Microsoft.Spark.Sql.Types;
using Microsoft.Spark.Extensions.Delta;
using static Microsoft.Spark.Sql.Functions;

Environment.SetEnvironmentVariable("EVENT_HUB_CONNECTION_STRING", eventHubConnectionStringWithSharedAccessKeyAndEntityPath);
var eventHubConnectionString = Environment.GetEnvironmentVariable("EVENT_HUB_CONNECTION_STRING");
string bootstrapServers = $"{eventHubsNamespace}.servicebus.windows.net:9093"; // 9093 is the port used to communicate with Event Hubs, see [troubleshooting guide](https://docs.microsoft.com/azure/event-hubs/troubleshooting-guide)
string eh_sasl = $"org.apache.kafka.common.security.plain.PlainLoginModule required username=\"$ConnectionString\" password=\"{eventHubConnectionString}\";";

var path = $"abfss://{delta_lake_container_name}@{storageAccountName}.dfs.core.windows.net/{blobName}";
var spark = SparkSession.Builder().Config($"fs.azure.account.key.{storageAccountName}.dfs.core.windows.net", storageAccountKey).GetOrCreate();

Console.WriteLine(eh_sasl);

var streamingDf = spark
    .ReadStream()
    .Format("kafka")
    .Option("kafka.bootstrap.servers", bootstrapServers)
    .Option("subscribe", eventHubsInstance)
    .Option("kafka.sasl.mechanism", "PLAIN")
    .Option("kafka.security.protocol", "SASL_SSL")
    .Option("kafka.sasl.jaas.config", eh_sasl)
    .Option("kafka.request.timeout.ms", "60000")
    .Option("kafka.session.timeout.ms", "60000")
    //.Option("failOnDataLoss", "false")
    //.Option("checkpointLocation", "/tmp/kafka_cp.txt")
    .Load()
    .WriteStream()
    .ForeachBatch((df, id) =>
    {
        Console.WriteLine($"batch id: {id}");
        df.Show();
        if(df.Head(1).Any()){

            df = df
            .WithColumn("year", Functions.Year(df["timestamp"]))
            .WithColumn("month", Functions.Month(df["timestamp"]))
            .WithColumn("day", Functions.DayOfMonth(df["timestamp"]));
            df
            .Write()
            .PartitionBy("year", "month", "day")
            .Format("delta")
            .Mode(SaveMode.Append)
            .Save(path); //"/tmp/delta-table"
        }
    })
    .Start();

streamingDf.AwaitTermination();

Installed Packages Microsoft.Spark, 2.1.0 Microsoft.Spark.Extensions.Delta, 2.1.0

Loading extensions from `Microsoft.Data.Analysis.Interactive.dll`

[2022-03-09T15:34:16.3800482Z] [a0bba748f140] [Info] [ConfigurationService] 'DOTNETBACKEND_PORT' environment variable is not set.
[2022-03-09T15:34:16.3821336Z] [a0bba748f140] [Info] [ConfigurationService] Using port 5567 for connection.
[2022-03-09T15:34:16.3828165Z] [a0bba748f140] [Info] [JvmBridge] JvMBridge port is 5567
[2022-03-09T15:34:16.3833600Z] [a0bba748f140] [Info] [JvmBridge] The number of JVM backend thread is set to 10. The max number of concurrent sockets in JvmBridge is set to 7.
org.apache.kafka.common.security.plain.PlainLoginModule required username="$ConnectionString" password="Endpoint=sb://evhns-timeseries-timeseries-u-001.servicebus.windows.net/;SharedAccessKeyName=listen;SharedAccessKey=lGW5ENgWGFeRhNNBhPkq4uAYrebkWhSTa0ZbvafB9cw=;EntityPath=received-timeseries";
[2022-03-09T15:34:20.8633921Z] [a0bba748f140] [Info] [CallbackServer] Starting CallbackServer.
[2022-03-09T15:34:20.8908757Z] [a0bba748f140] [Info] [CallbackServer] Started CallbackServer on 127.0.0.1:4

Error: System.Exception: JVM method execution failed: Nonstatic method 'awaitTermination' failed for class '19' when called with no arguments
 ---> Microsoft.Spark.JvmException: org.apache.spark.sql.streaming.StreamingQueryException: Error while verifying end of stream.
=== Streaming Query ===
Identifier: [id = 12153932-763b-4315-95ac-5ffd0ccaf453, runId = 86b8bdea-bd7a-42f5-a7b7-7dc858c826f7]
Current Committed Offsets: {KafkaV2[Subscribe[received-timeseries]]: {"received-timeseries":{"2":26,"1":27,"3":35,"0":29}}}
Current Available Offsets: {KafkaV2[Subscribe[received-timeseries]]: {"received-timeseries":{"2":26,"1":27,"3":36,"0":29}}}

Current State: ACTIVE
Thread State: RUNNABLE

Logical Plan:
StreamingDataSourceV2Relation [key#7, value#8, topic#9, partition#10, offset#11L, timestamp#12, timestampType#13], org.apache.spark.sql.kafka010.KafkaSourceProvider$KafkaScan@2b95e02e, KafkaV2[Subscribe[received-timeseries]]

	at org.apache.spark.sql.execution.streaming.StreamExecution.org$apa